<a href="https://colab.research.google.com/github/tejaspradhan/Graph-Neural-Networks/blob/main/TextGCNs/GNN_Humour_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>